In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasetraw2/RawData.xlsx


In [5]:
import numpy as np
import pandas as pd

# ==== Ajusta solo esta ruta a tu dataset en Kaggle ====
# Ejemplo si tu dataset se llama "datos-cla":
# /kaggle/input/datos-cla/RawData.xlsx
INPUT_FILE = "/kaggle/input/datasetraw2/RawData.xlsx"
INPUT_SHEET = 0
OUTPUT_CSV  = "/kaggle/working/salida_filtrada.csv"

# ===== Utilidades para localizar columnas por nombre con acentos/espacios =====
def _norm(s: str) -> str:
    import unicodedata, re
    s = str(s).strip().lower()
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    s = re.sub(r'\s+', ' ', s)
    return s

def find_col(df: pd.DataFrame, target_name: str):
    target_norm = _norm(target_name)
    for c in df.columns:
        if _norm(c) == target_norm:
            return c
    # fallback: coincidencia parcial
    candidates = [c for c in df.columns if target_norm in _norm(c)]
    if not candidates:
        raise KeyError(f"No se encontró la columna '{target_name}'. Disponibles: {list(df.columns)}")
    # prioriza prefijo igual, si existe
    pref = [c for c in candidates if _norm(c).startswith(target_norm)]
    return (pref[0] if pref else candidates[0])

def get_col_range(df: pd.DataFrame, start_col: str, end_col: str):
    """
    Devuelve la lista de columnas en orden izquierda→derecha entre start_col y end_col (inclusive).
    Evita problemas si el usuario pasa los nombres invertidos.
    """
    cols_all = list(df.columns)
    if start_col not in df.columns or end_col not in df.columns:
        raise KeyError("start_col o end_col no existen en el DataFrame.")
    i1, i2 = cols_all.index(start_col), cols_all.index(end_col)
    lo, hi = (i1, i2) if i1 <= i2 else (i2, i1)
    return cols_all[lo:hi+1]



In [6]:
# Carga del archivo
df_raw = pd.read_excel(INPUT_FILE, sheet_name=INPUT_SHEET)
print(df_raw.loc[0, "MES -33"])  # debería ser '12,67'
print(type(df_raw.loc[0, "MES -33"]))
print(df_raw.columns.tolist()[:10])   # primeras columnas
print(df_raw.columns.tolist()[-10:])  # últimas columnas

12.67
<class 'numpy.float64'>
['OT', 'Plan de facturacion', 'Estrato', 'MES -36', 'MES -35', 'MES -34', 'MES -33', 'MES -32', 'MES -31', 'MES -30']
['MES -9', 'MES -8', 'MES -7', 'MES -6', 'MES -5', 'MES -4', 'MES -3', 'MES -2', 'MES -1', 'Resultado']


In [7]:

# === FUNCIONES ESTADÍSTICAS ANTES/DESPUÉS DE CAÍDA ===

def _norm_value(x):
    try:
        s = str(x).strip().replace(",", ".")
        return float(s)
    except:
        return np.nan

def calcular_estadistica(row, df, col_inicio, col_fin, funcion):
    try:
        cols = get_col_range(df, col_inicio, col_fin)
        vals = pd.to_numeric(row[cols].apply(_norm_value), errors='coerce')
        if vals.count() == 0:
            return "Sin Caida"
        return funcion(vals)
    except:
        return "Sin Caida"

def promedio_antes_caida(row, df, col_inicio, col_inicio_caida):
    if pd.isna(col_inicio_caida) or col_inicio_caida == "sin caida":
        return "Sin Caida"
    return calcular_estadistica(row, df, col_inicio, col_inicio_caida, np.nanmean)

def desviacion_antes_caida(row, df, col_inicio, col_inicio_caida):
    if pd.isna(col_inicio_caida) or col_inicio_caida == "sin caida":
        return "Sin Caida"
    return calcular_estadistica(row, df, col_inicio, col_inicio_caida, np.nanstd)

def maximo_antes_caida(row, df, col_inicio, col_inicio_caida):
    if pd.isna(col_inicio_caida) or col_inicio_caida == "sin caida":
        return "Sin Caida"
    return calcular_estadistica(row, df, col_inicio, col_inicio_caida, np.nanmax)

def minimo_antes_caida(row, df, col_inicio, col_inicio_caida):
    if pd.isna(col_inicio_caida) or col_inicio_caida == "sin caida":
        return "Sin Caida"
    return calcular_estadistica(row, df, col_inicio, col_inicio_caida, np.nanmin)

def promedio_despues_caida(row, df, col_fin_caida, col_final):
    if pd.isna(col_fin_caida) or col_fin_caida == "sin caida":
        return "Sin Caida"
    return calcular_estadistica(row, df, col_fin_caida, col_final, np.nanmean)

def desviacion_despues_caida(row, df, col_fin_caida, col_final):
    if pd.isna(col_fin_caida) or col_fin_caida == "sin caida":
        return "Sin Caida"
    return calcular_estadistica(row, df, col_fin_caida, col_final, np.nanstd)

def maximo_despues_caida(row, df, col_fin_caida, col_final):
    if pd.isna(col_fin_caida) or col_fin_caida == "sin caida":
        return "Sin Caida"
    return calcular_estadistica(row, df, col_fin_caida, col_final, np.nanmax)

def minimo_despues_caida(row, df, col_fin_caida, col_final):
    if pd.isna(col_fin_caida) or col_fin_caida == "sin caida":
        return "Sin Caida"
    return calcular_estadistica(row, df, col_fin_caida, col_final, np.nanmin)

def contar_consumos_cero(row, df, col_inicio, col_fin):
    try:
        cols = get_col_range(df, col_inicio, col_fin)
        vals = pd.to_numeric(row[cols].apply(_norm_value), errors='coerce')
        return int((vals == 0).sum())
    except:
        return np.nan

def contar_consumos_por_promedio(row, df, col_inicio, col_fin):
    try:
        cols = get_col_range(df, col_inicio, col_fin)
        vals = pd.to_numeric(row[cols].apply(_norm_value), errors='coerce')
        return int(vals.apply(lambda x: not pd.isna(x) and not float(x).is_integer()).sum())
    except:
        return np.nan


In [8]:


# Columnas tal como vienen en tu archivo
col_OT         = find_col(df_raw, "OT")
col_plan       = find_col(df_raw, "Plan de facturacion")
col_estrato    = find_col(df_raw, "Estrato")
col_resultado  = find_col(df_raw, "Resultado")

# Construir dataframe mínimo con 'Resultado' al final
df_out = pd.DataFrame({
    col_OT: df_raw[col_OT],
    col_plan: df_raw[col_plan],
    col_estrato: df_raw[col_estrato],
})
df_out[col_resultado] = df_raw[col_resultado]

# Exportar
df_out.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print("CSV creado en:", OUTPUT_CSV)
df_out.head()


CSV creado en: /kaggle/working/salida_filtrada.csv


,OT,Plan de facturacion,Estrato,Resultado
0,4562742,Plan 204,Estrato 3,Fraude
1,4685551,Plan 204,Estrato 2,Fraude
2,4630304,Plan 123,Estrato -1,Fraude
3,4647545,Plan 123,Estrato -1,Fraude
4,5551363,Plan 204,Estrato 3,Fraude


In [9]:
def _slice_row_values(row: pd.Series, df: pd.DataFrame, start_col: str, end_col: str):
    cols = get_col_range(df, start_col, end_col)
    vals = row[cols].values.astype(float)
    return vals, cols

def calcular_promedio(row: pd.Series, df: pd.DataFrame, start_col: str, end_col: str) -> float:
    vals, _ = _slice_row_values(row, df, start_col, end_col)
    return float(np.nanmean(vals)) if len(vals) else np.nan

def desviacion_estandar(row: pd.Series, df: pd.DataFrame, start_col: str, end_col: str) -> float:
    vals, _ = _slice_row_values(row, df, start_col, end_col)
    return float(np.nanstd(vals, ddof=1)) if len(vals) > 1 else 0.0

def maximo(row: pd.Series, df: pd.DataFrame, start_col: str, end_col: str) -> float:
    vals, _ = _slice_row_values(row, df, start_col, end_col)
    return float(np.nanmax(vals)) if len(vals) else np.nan

def minimo(row: pd.Series, df: pd.DataFrame, start_col: str, end_col: str) -> float:
    vals, _ = _slice_row_values(row, df, start_col, end_col)
    return float(np.nanmin(vals)) if len(vals) else np.nan


In [11]:
import numpy as np
import pandas as pd

def determinar_caida(row: pd.Series,
                     df: pd.DataFrame,
                     start_col: str,
                     end_col: str,
                     umbral_caida: float = 0.10,
                     consecutivos: int = 3):
    """
    Detecta MULTIPLES caídas y devuelve SIEMPRE una tupla:
      (col_inicio_ultima, col_fin_ultima, pasos_ultima, num_caidas)

    - La "caída" se define como racha de 'consecutivos' valores (o más) <= (1 - umbral_caida)*promedio,
      donde el promedio inicial de cada segmento es el de sus 2 primeras columnas.
    - Mientras una racha esté activa, el promedio NO cambia.
    - Si la racha no alcanza 'consecutivos', se expande la ventana de promedio hasta la columna actual.
    - Al CERRAR una caída (porque aparece un valor que NO cumple), se:
        * registra esa caída,
        * se reinicia el promedio tomando como nuevas 2 primeras columnas: (último de la caída, valor que no cumplió),
        * y se continúa la búsqueda desde la 3ra columna del nuevo segmento.
    - Si una caída llega hasta el final del rango, también se registra.
    """

    cols = get_col_range(df, start_col, end_col)

    # 1) Normalizar coma decimal → punto y convertir a numérico de forma segura
    def _norm(x):
        if pd.isna(x):
            return np.nan
        s = str(x).strip().replace(",", ".")
        if s == "" or s == "-" or s.lower() == "nan":
            return np.nan
        try:
            return float(s)
        except Exception:
            return np.nan

    vals = row[cols].apply(_norm).values.astype(float)
    n = len(vals)
    if n < 3:
        return (None, None, 0, 0)

    # --- Segmentación para permitir reinicios del promedio ---
    base_start = 0       # índice donde inicia el "segmento" actual (para promediar desde aquí)
    window_end = base_start + 1  # las primeras 2 columnas del segmento
    if window_end >= n:
        return (None, None, 0, 0)

    def _avg_segment(beg: int, end_inclusive: int) -> float:
        """Promedio en [beg, end_inclusive] ignorando NaN; si todo es NaN -> 0.0 (como VBA)."""
        window = vals[beg:end_inclusive + 1]
        mask = ~np.isnan(window)
        if mask.any():
            return float(np.mean(window[mask]))
        return 0.0

    avg = _avg_segment(base_start, window_end)

    i = base_start + 2   # la comparación empieza en la 3ra columna del segmento
    racha = 0
    inicio_racha_idx = None
    caida_confirmada = False

    # Acumuladores de resultado (guardaremos la ÚLTIMA caída encontrada)
    num_caidas = 0
    last_ini_idx = None
    last_fin_idx = None
    last_pasos = 0

    while i < n:
        v = vals[i]
        limite = (1 - umbral_caida) * avg
        # NaN -> no cumple (como en VBA para no numéricos)
        cumple = (not np.isnan(v)) and (v <= limite)

        if cumple:
            racha += 1
            if racha == 1:
                inicio_racha_idx = i
            if racha >= consecutivos:
                caida_confirmada = True
            i += 1
            continue

        # NO cumple
        if caida_confirmada:
            # Cerrar caída justo antes de este valor i
            col_ini = cols[inicio_racha_idx]
            col_fin = cols[i - 1]
            pasos = (i - 1) - inicio_racha_idx + 1

            # Actualizar "última" caída y contador
            last_ini_idx = inicio_racha_idx
            last_fin_idx = i - 1
            last_pasos = pasos
            num_caidas += 1

            # Reiniciar SEGMENTO con dos primeras columnas: (último de la caída, este valor que no cumplió)
            base_start = i - 1       # índice del último de la caída
            if base_start + 1 >= n:  # no hay 2 columnas para formar nuevo promedio
                break
            window_end = base_start + 1
            avg = _avg_segment(base_start, window_end)

            # Reiniciar estado de racha y seguir comparando desde la 3ra col del nuevo segmento
            racha = 0
            inicio_racha_idx = None
            caida_confirmada = False
            i = base_start + 2
            continue
        else:
            # No se alcanzó racha => expandir ventana del segmento actual
            window_end = i
            avg = _avg_segment(base_start, window_end)
            racha = 0
            inicio_racha_idx = None
            i += 1
            continue

    # Fin del while: si quedó caída abierta (confirmada) hasta el final, cerrarla
    if caida_confirmada and inicio_racha_idx is not None:
        col_ini = cols[inicio_racha_idx]
        col_fin = cols[n - 1]
        pasos = (n - 1) - inicio_racha_idx + 1

        last_ini_idx = inicio_racha_idx
        last_fin_idx = n - 1
        last_pasos = pasos
        num_caidas += 1

    # Preparar retorno: última caída o "no hay"
    if num_caidas > 0 and last_ini_idx is not None:
        return (cols[last_ini_idx], cols[last_fin_idx], int(last_pasos), int(num_caidas))
    else:
        return (None, None, 0, 0)




In [12]:
OUTPUT_CSV_EXT = "/kaggle/working/salida_extendida.csv"
COL_INICIO_CONS = "MES -36"
COL_FIN_CONS = "MES -1"
# 1) Agregados sobre el rango MES -36 ... MES -1 (si no los tienes aún, se recalculan)
df_raw["promedio_36m"] = df_raw.apply(lambda r: calcular_promedio(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)
df_raw["desv_36m"]     = df_raw.apply(lambda r: desviacion_estandar(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)
df_raw["max_36m"]      = df_raw.apply(lambda r: maximo(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)
df_raw["min_36m"]      = df_raw.apply(lambda r: minimo(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)

COL_INICIO_CONS = "MES -6"
df_raw["promedio_6m_1m"] = df_raw.apply(lambda r: calcular_promedio(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)
df_raw["desv_6m_1m"]     = df_raw.apply(lambda r: desviacion_estandar(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)
df_raw["max_6m_1m"]      = df_raw.apply(lambda r: maximo(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)
df_raw["min_6m_1m"]      = df_raw.apply(lambda r: minimo(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)


In [13]:
COL_INICIO_CONS = "MES -12"
COL_FIN_CONS = "MES -6"
# 1) Agregados sobre el rango MES -36 ... MES -1 (si no los tienes aún, se recalculan)
df_raw["promedio_12m_6m"] = df_raw.apply(lambda r: calcular_promedio(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)
df_raw["desv_12m_6m"]     = df_raw.apply(lambda r: desviacion_estandar(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)
df_raw["max_12m_6m"]      = df_raw.apply(lambda r: maximo(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)
df_raw["min_12m_6m"]      = df_raw.apply(lambda r: minimo(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)

COL_INICIO_CONS = "MES -24"
COL_FIN_CONS = "MES -12"
df_raw["promedio_24m_12m"] = df_raw.apply(lambda r: calcular_promedio(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)
df_raw["desv_24m_12m"]     = df_raw.apply(lambda r: desviacion_estandar(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)
df_raw["max_24m_12m"]      = df_raw.apply(lambda r: maximo(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)
df_raw["min_24m_12m"]      = df_raw.apply(lambda r: minimo(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)


In [14]:
COL_INICIO_CONS = "MES -36"
COL_FIN_CONS = "MES -24"
df_raw["promedio_36m_24m"] = df_raw.apply(lambda r: calcular_promedio(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)
df_raw["desv_36m_24m"]     = df_raw.apply(lambda r: desviacion_estandar(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)
df_raw["max_36m_24m"]      = df_raw.apply(lambda r: maximo(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)
df_raw["min_36m_24m"]      = df_raw.apply(lambda r: minimo(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)


In [15]:
df_raw.head(5)

,OT,Plan de facturacion,Estrato,MES -36,MES -35,MES -34,MES -33,MES -32,MES -31,MES -30,...,max_12m_6m,min_12m_6m,promedio_24m_12m,desv_24m_12m,max_24m_12m,min_24m_12m,promedio_36m_24m,desv_36m_24m,max_36m_24m,min_36m_24m
0,4562742,Plan 204,Estrato 3,7.0,20.0,18.0,12.67,10.2,14.6,26.0,...,29.0,24.0,25.538462,2.569546,31.0,23.0,19.190000,6.287551,26.0,7.0
1,4685551,Plan 204,Estrato 2,5.0,4.0,4.0,4.00,6.0,5.0,7.0,...,12.0,4.0,7.769231,4.675358,19.0,3.0,4.846154,0.987096,7.0,4.0
2,4630304,Plan 123,Estrato -1,7.0,6.0,7.0,6.00,6.0,9.0,8.0,...,19.0,3.0,12.538462,3.843076,19.0,7.0,8.720769,4.037071,19.0,6.0
3,4647545,Plan 123,Estrato -1,465.0,606.0,564.0,585.00,621.0,642.0,603.6,...,312.0,270.0,363.897692,171.994518,621.0,162.0,596.201538,59.328560,711.0,465.0
4,5551363,Plan 204,Estrato 3,14.0,13.0,14.0,13.00,12.0,12.0,12.0,...,16.0,12.0,12.230769,1.480644,15.0,9.0,13.461538,1.050031,15.0,12.0


In [16]:
COL_INICIO_CONS = "MES -36"
COL_FIN_CONS = "MES -1"
def _split_caida_multiple(res):
    try:
        col_ini, col_fin, pasos, n_caidas = res
    except Exception:
        col_ini, col_fin, pasos, n_caidas = (None, None, 0, 0)
    return pd.Series({
        "Caida_Inicio":  col_ini if col_ini is not None else "sin caida",
        "Caida_Fin":     col_fin if col_fin is not None else "sin caida",
        "LongitudCaida": int(pasos) if pd.notna(pasos) else 0,
        "CantidadCaidas": int(n_caidas) if pd.notna(n_caidas) else 0
    })

caidas = (
    df_raw
    .apply(lambda r: determinar_caida(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS,
                                      umbral_caida=0.10, consecutivos=3),
           axis=1)
    .apply(_split_caida_multiple)
)

# Evitar duplicados si ya existen
for c in ["Caida_Inicio", "Caida_Fin", "LongitudCaida", "CantidadCaidas"]:
    if c in df_raw.columns:
        df_raw.drop(columns=[c], inplace=True)

df_raw = pd.concat([df_raw, caidas], axis=1)


# 3) Ensamblar DataFrame extendido (Resultado al final)
cols_ext = [
    col_OT, col_plan, col_estrato, "promedio_36m", "desv_36m", "max_36m", "min_36m",
    "promedio_6m_1m", "desv_6m_1m", "max_6m_1m", "min_6m_1m",
    "promedio_12m_6m", "desv_12m_6m", "max_12m_6m", "min_12m_6m",
    "promedio_24m_12m", "desv_24m_12m", "max_24m_12m", "min_24m_12m",
    "promedio_36m_24m", "desv_36m_24m", "max_36m_24m", "min_36m_24m",
    "Caida_Inicio", "Caida_Fin", "LongitudCaida","CantidadCaidas"
]
df_ext = df_raw[cols_ext].copy()
df_ext[col_resultado] = df_raw[col_resultado]  # dejar Resultado al final

# 4) Exportar
df_ext.to_csv(OUTPUT_CSV_EXT, index=False, encoding="utf-8-sig")
print("CSV extendido creado en:", OUTPUT_CSV_EXT)
df_ext.head()



CSV extendido creado en: /kaggle/working/salida_extendida.csv


,OT,Plan de facturacion,Estrato,promedio_36m,desv_36m,max_36m,min_36m,promedio_6m_1m,desv_6m_1m,max_6m_1m,...,min_24m_12m,promedio_36m_24m,desv_36m_24m,max_36m_24m,min_36m_24m,Caida_Inicio,Caida_Fin,LongitudCaida,CantidadCaidas,Resultado
0,4562742,Plan 204,Estrato 3,21.068611,6.991463,31.0,2.0,12.500000,7.943551,24.00,...,23.0,19.190000,6.287551,26.0,7.0,MES -5,MES -1,5,1,Fraude
1,4685551,Plan 204,Estrato 2,7.194444,3.519898,19.0,3.0,9.833333,1.722401,12.00,...,3.0,4.846154,0.987096,7.0,4.0,sin caida,sin caida,0,0,Fraude
2,4630304,Plan 123,Estrato -1,9.710556,4.449164,19.0,3.0,8.535000,4.452342,13.96,...,7.0,8.720769,4.037071,19.0,6.0,MES -10,MES -6,5,1,Fraude
3,4647545,Plan 123,Estrato -1,407.619167,185.565276,711.0,69.0,214.500000,78.660664,282.00,...,162.0,596.201538,59.328560,711.0,465.0,MES -20,MES -1,20,1,Fraude
4,5551363,Plan 204,Estrato 3,13.027778,1.463579,16.0,9.0,13.666667,1.632993,16.00,...,9.0,13.461538,1.050031,15.0,12.0,MES -23,MES -16,8,2,Fraude


In [17]:

# == APLICAR FUNCIONES ESTADÍSTICAS AL DATAFRAME ==
# Asumiendo que el DataFrame principal es df_raw
# Y que ya contiene columnas: Caida_Inicio, Caida_Fin

COL_INICIO_CONS = "MES -36"
COL_FIN_CONS    = "MES -1"

df_raw["Prom_AntesCaida"]       = df_raw.apply(lambda r: promedio_antes_caida(r, df_raw, COL_INICIO_CONS, r["Caida_Inicio"]), axis=1)
df_raw["Desv_AntesCaida"]       = df_raw.apply(lambda r: desviacion_antes_caida(r, df_raw, COL_INICIO_CONS, r["Caida_Inicio"]), axis=1)
df_raw["Max_AntesCaida"]        = df_raw.apply(lambda r: maximo_antes_caida(r, df_raw, COL_INICIO_CONS, r["Caida_Inicio"]), axis=1)
df_raw["Min_AntesCaida"]        = df_raw.apply(lambda r: minimo_antes_caida(r, df_raw, COL_INICIO_CONS, r["Caida_Inicio"]), axis=1)

df_raw["Prom_DespuesCaida"]     = df_raw.apply(lambda r: promedio_despues_caida(r, df_raw, r["Caida_Fin"], COL_FIN_CONS), axis=1)
df_raw["Desv_DespuesCaida"]     = df_raw.apply(lambda r: desviacion_despues_caida(r, df_raw, r["Caida_Fin"], COL_FIN_CONS), axis=1)
df_raw["Max_DespuesCaida"]      = df_raw.apply(lambda r: maximo_despues_caida(r, df_raw, r["Caida_Fin"], COL_FIN_CONS), axis=1)
df_raw["Min_DespuesCaida"]      = df_raw.apply(lambda r: minimo_despues_caida(r, df_raw, r["Caida_Fin"], COL_FIN_CONS), axis=1)

df_raw["ConsumosCero"]          = df_raw.apply(lambda r: contar_consumos_cero(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)
df_raw["ConsumosDecimal"]       = df_raw.apply(lambda r: contar_consumos_por_promedio(r, df_raw, COL_INICIO_CONS, COL_FIN_CONS), axis=1)


In [18]:

# === ENSAMBLAR DATASET EXTENDIDO CON NUEVAS COLUMNAS Y EXPORTAR ===

cols_ext = [
    col_OT, col_plan, col_estrato,
    "promedio_36m", "desv_36m", "max_36m", "min_36m",
    "promedio_6m_1m", "desv_6m_1m", "max_6m_1m", "min_6m_1m",
    "promedio_12m_6m", "desv_12m_6m", "max_12m_6m", "min_12m_6m",
    "promedio_24m_12m", "desv_24m_12m", "max_24m_12m", "min_24m_12m",
    "promedio_36m_24m", "desv_36m_24m", "max_36m_24m", "min_36m_24m",
    "Caida_Inicio", "Caida_Fin", "LongitudCaida", "CantidadCaidas",
    "Prom_AntesCaida", "Desv_AntesCaida", "Max_AntesCaida", "Min_AntesCaida",
    "Prom_DespuesCaida", "Desv_DespuesCaida", "Max_DespuesCaida", "Min_DespuesCaida",
    "ConsumosCero", "ConsumosDecimal"
]

df_ext = df_raw[cols_ext].copy()
df_ext[col_resultado] = df_raw[col_resultado]  # mantener resultado al final

df_ext.to_csv(OUTPUT_CSV_EXT, index=False, encoding="utf-8-sig")
print("CSV extendido creado en:", OUTPUT_CSV_EXT)
df_ext.head()


CSV extendido creado en: /kaggle/working/salida_extendida.csv


,OT,Plan de facturacion,Estrato,promedio_36m,desv_36m,max_36m,min_36m,promedio_6m_1m,desv_6m_1m,max_6m_1m,...,Desv_AntesCaida,Max_AntesCaida,Min_AntesCaida,Prom_DespuesCaida,Desv_DespuesCaida,Max_DespuesCaida,Min_DespuesCaida,ConsumosCero,ConsumosDecimal,Resultado
0,4562742,Plan 204,Estrato 3,21.068611,6.991463,31.0,2.0,12.500000,7.943551,24.00,...,6.344569,31.0,2.0,6.0,0.0,6.0,6.0,0,3,Fraude
1,4685551,Plan 204,Estrato 2,7.194444,3.519898,19.0,3.0,9.833333,1.722401,12.00,...,Sin Caida,Sin Caida,Sin Caida,Sin Caida,Sin Caida,Sin Caida,Sin Caida,0,0,Fraude
2,4630304,Plan 123,Estrato -1,9.710556,4.449164,19.0,3.0,8.535000,4.452342,13.96,...,4.321272,19.0,4.0,8.535,4.064414,13.96,3.0,0,5,Fraude
3,4647545,Plan 123,Estrato -1,407.619167,185.565276,711.0,69.0,214.500000,78.660664,282.00,...,113.24374,711.0,166.86,243.0,0.0,243.0,243.0,0,7,Fraude
4,5551363,Plan 204,Estrato 3,13.027778,1.463579,16.0,9.0,13.666667,1.632993,16.00,...,1.505093,15.0,9.0,13.4375,1.321398,16.0,11.0,0,0,Fraude
